In [1]:
from astropy.table import Table

import glob

import numpy as np

In [17]:
master_table = Table.read('../master_file_vflag_10.txt', format='ascii.ecsv')

## Galaxies with rotation curves

In [18]:
boolean = np.logical_or(master_table['curve_used'] == 'non', master_table['curve_used'] == 'none')
#boolean = master_table['curve_used'] == -99

rotCurve_galaxies = master_table[np.logical_not(boolean)]

print('Galaxies in MaNGA DR15:', len(master_table))
print('Galaxies with rotation curves:', len(rotCurve_galaxies))

Galaxies in MaNGA DR15: 4815
Galaxies with rotation curves: 3371


## Galaxies in each environment

In [19]:
vboolean = rotCurve_galaxies['vflag'] == 1
wboolean = rotCurve_galaxies['vflag'] == 0

void_table = rotCurve_galaxies[vboolean]
wall_table = rotCurve_galaxies[wboolean]
edge_table = rotCurve_galaxies[np.logical_not(np.logical_or(vboolean, wboolean))]

print('Void galaxies with rotation curves:', len(void_table))
print('Wall galaxies with rotation curves:', len(wall_table))
print('Edge galaxies with rotation curves:', len(edge_table))

Void galaxies with rotation curves: 809
Wall galaxies with rotation curves: 1303
Edge galaxies with rotation curves: 1259


## Galaxies by magnitude

In [20]:
print('Dwarf void galaxies:', sum(void_table['rabsmag'] >= -17))
print('Dwarf wall galaxies:', sum(wall_table['rabsmag'] >= -17))

print('-17:-18 void galaxies:', sum( np.logical_and( void_table['rabsmag'] < -17, void_table['rabsmag'] >= -18)))
print('-17:-18 wall galaxies:', sum( np.logical_and( wall_table['rabsmag'] < -17, wall_table['rabsmag'] >= -18)))

print('-18:-19 void galaxies:', sum( np.logical_and( void_table['rabsmag'] < -18, void_table['rabsmag'] >= -19)))
print('-18:-19 wall galaxies:', sum( np.logical_and( wall_table['rabsmag'] < -18, wall_table['rabsmag'] >= -19)))

print('-19:-20 void galaxies:', sum( np.logical_and( void_table['rabsmag'] < -19, void_table['rabsmag'] >= -20)))
print('-19:-20 wall galaxies:', sum( np.logical_and( wall_table['rabsmag'] < -19, wall_table['rabsmag'] >= -20)))

print('Void galaxies brighter than -20:', sum( void_table['rabsmag'] < -20))
print('Wall galaxies brighter than -20:', sum( wall_table['rabsmag'] < -20))

Dwarf void galaxies: 9
Dwarf wall galaxies: 11
-17:-18 void galaxies: 92
-17:-18 wall galaxies: 121
-18:-19 void galaxies: 190
-18:-19 wall galaxies: 262
-19:-20 void galaxies: 191
-19:-20 wall galaxies: 301
Void galaxies brighter than -20: 327
Wall galaxies brighter than -20: 608


## Fraction of masked spaxels

In [17]:
sum_frac_masked_spaxels = 0
N_gal = 0

for file in glob.iglob('../rot_curve_data_files/*_gal_stat_data.txt'):
    
    # Extract plate, fiberID
    file_name = file[24:]
    components = file_name.split('_')
    plate, fiberID = components[0].split('-')
    
    # Find galaxy in master_table
    plate_indices = np.argwhere(master_table['MaNGA_plate'] == int(plate))
    fiber_idx = np.where(master_table['MaNGA_fiberID'][plate_indices] == int(fiberID))
    gal_idx = plate_indices[fiber_idx]
    
    if master_table['curve_used'][gal_idx] != 'non' and master_table['curve_used'][gal_idx] != 'none':
        
        # Read in data
        gal_stats = Table.read(file, format='ascii.ecsv')

        # Update number of galaxies
        N_gal += 1

        # Update sum of the fraction of the number of masked spaxels
        sum_frac_masked_spaxels += gal_stats['frac_masked_spaxels'][0]
        
        
print('Average number of masked spaxels:', sum_frac_masked_spaxels/N_gal)

Average number of masked spaxels: 0.212001870176713
